# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [14]:
# imports
# Core Python utilities
import os
import json
import requests
from IPython.display import Markdown, display, update_display


# Environment variable loading
from dotenv import load_dotenv

# OpenAI client
from openai import OpenAI


In [15]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [16]:
# set up environment
# set up environment

load_dotenv(override=True)

# Load OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY and OPENAI_API_KEY.startswith("sk-") and len(OPENAI_API_KEY) > 10:
    print("✅ OpenAI API key loaded successfully")
else:
    print("⚠️ OpenAI API key missing or invalid. Check your .env file.")

# Initialise OpenAI client
openai = OpenAI(api_key=OPENAI_API_KEY)


✅ OpenAI API key loaded successfully


In [17]:
SYSTEM_PROMPT = """
You are a friendly technical tutor.
Explain the answer clearly in 3 parts:
1) What it does
2) Why it works
3) Common pitfalls / edge cases
Keep it concise but helpful. Use small examples if useful.
"""

In [18]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [19]:
# Get gpt-4o-mini to answer, with streaming
def explain_with_openai_streaming(question_text: str):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question_text.strip()},
        ],
        stream=True,
    )

    response_text = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta and getattr(delta, "content", None):
            response_text += delta.content
            update_display(Markdown(response_text), display_id=display_handle.display_id)

    return response_text

In [20]:
# Get Llama 3.2 to answer
def explain_with_ollama(question_text: str, model: str = MODEL_LLAMA):
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question_text.strip()},
        ],
        "stream": False,
    }

    r = requests.post("http://localhost:11434/api/chat", json=payload, timeout=120)
    r.raise_for_status()
    return r.json()["message"]["content"]

In [22]:
print("=== OpenAI (streaming) ===")
_ = explain_with_openai_streaming(question)



=== OpenAI (streaming) ===


Sure! Let's break down the code you provided step-by-step:

### 1) What it does
This code uses a generator expression to yield each unique author's name from a collection of `books`. Specifically:
- It goes through each `book` in the `books` iterable (which is presumably a list of dictionaries).
- For each `book`, it tries to retrieve the value associated with the key `"author"` using `book.get("author")`.
- It includes the author in the result only if the author is not `None` or an empty string (due to the condition `if book.get("author")`).
- `yield from` causes the generator to yield each value produced by the included generator expression, which only gets unique authors due to the use of a set.

### 2) Why it works
- **Generator Expression**: `{book.get("author") for book in books if book.get("author")}` creates a set of authors, effectively filtering out duplicates and excluding any non-existent authors. The set comprehension evaluates to a set of author names.
- **`yield from`**: This keyword allows values produced from the set (which includes unique authors) to be yielded one by one within a generator function. This means each time the generator is iterated over, it will return the next author until all have been processed.

### 3) Common pitfalls / edge cases
- **Empty or Missing "author" Keys**: If the `books` list is empty or if none of the books have the "author" key, the output will simply yield nothing.
  
   ```python
   books = []
   # No authors to yield, will produce no output.
   ```
   
- **Non-string Author Values**: If `book.get("author")` returns values that are not strings (e.g., `None`, numbers, or objects), these values may affect what gets included in the set.
  
   ```python
   books = [{"author": None}, {"author": 42}, {"author": "Author A"}]
   # Only "Author A" will be yielded.
   ```

- **Mutability of Set**: If for any reason the contents of `books` change while iterating, it may lead to unexpected results if the generator is reused. 

- **Performance**: If `books` contains a large number of entries, the performance could be affected due to set uniqueness checks, though this is typically not noticeable in most practical scenarios.

In summary, this code efficiently filters and yields unique author names from a collection of books while taking care to exclude any missing or invalid entries!

In [ ]:
print("\n\n=== Ollama (llama3.2) ===")
ollama_answer = explain_with_ollama(question)
display(Markdown(ollama_answer))